In [1]:
from math import ceil, pi, e
from sympy import Symbol, diff, factorial, fraction, simplify, solve, symbols, ln, Rational
from sympy.functions import sin, cos, tan
from termcolor import colored
from sympy.parsing.sympy_parser import parse_expr

In [2]:
def checkint(floatt):
  floatt = float(floatt)
  if floatt%1!=0:
    return floatt
  else:
    return int(floatt)

def truncate(floatt,n=4):
  floatt = str(float(floatt))
  indx = floatt.find('.')
  if indx!=-1:
    try:
      return checkint(floatt[:indx+n+1])
    except:
      return checkint(floatt)
  else:
    return checkint(floatt)

def printt(string,color="red",end='\n'):
  print(colored(string,color),end=end)

def printmtrx(mtrx,ts,symbs):
  printt('\nt',end='\t')
  [printt(i,end='\t') for i in ts]
  print()
  for i in range(len(symbs)):
    printt(symbs[i],end='\t')
    for j in range(len(ts)):
      print(mtrx[j][i],end='\t')
    print()

In [11]:
def makefunc(returnn,flag):
  if flag:
    def _func(values):
      return values.get(returnn), returnn
  else:
    def _func(values):
      return parse_expr(returnn).subs(values), returnn
  return _func

def printinitials(a,b,h,length,N):
  ts = [a]
  print(f'h = (b-a)/N =>\tN = ({b}-{a})/{h} = {N}')
  for i in range(N):
    ts.append(truncate(ts[-1]+h))
  [print(f't{i} = {ts[i]}',end=' , ') for i in range(len(ts))]
  print()
  printt(f"{symbs[-1]}' = {fx}\n")

  us = [t]
  functions = list()

  for i in range(length):
    apos = "'"*i
    print(f'u{i}(t) = y{apos}(t)', end=' , ')
  print()
  for i in range(length):
    apos = "'"*i
    print(f'y{apos} = u{i+1}', end=' , ')
    us.append(f'u{i+1}')
  print()

  fy = str(fx)
  for i in range(length-1,-1,-1):
    apos = "'"*i
    fy = fy.replace(f'y{apos}',f'u{i+1}')
  print(f"u{length}' = {fy}\n")
  

  for i in range(1,length+1):
    if i!=length:
      print(f'f{i}{us} = {us[i+1]}')
      functions.append(makefunc(us[i+1],True))
    else:
      print(f'f{i}{us} = {fy}\n')
      functions.append(makefunc(fy,False))

  for i in range(1,length+1):
    print(f'u{i},0 = u{i}(t0) = u{i}({values[0]}) = {values[1][i-1]}')

  mtrx = list()
  for j in range(N+1):
    mtrx.append([0]*length)
  mtrx[0] = values[1]

  return ts, us, fy, mtrx, functions

def printmaineq(length):
  printt('\nui,j+1  = ui,j + (h/6)*(k1,i + 2(k2,i+k3,i) + k4,i)',color='blue')
  printt('where k1,i = fi(tj',end='',color='blue')
  for i in range(length):printt(f'; u{i+1},j ',color='blue',end='')
  for j in range(1,3):
    printt(f')\n      k{j+1},i = fi(tj+h/2',color='blue',end='')
    for i in range(length):printt(f'; u{i+1},j+(h/2)*k{j},{i+1} ',color='blue',end='')
  printt(f')\n      k4,i = fi(tj+h/2',color='blue',end='')
  for i in range(length):printt(f', u{i+1},j+h*k3,{i+1} ',color='blue',end='')
  printt(')\n',color='blue')

In [12]:
def getks(values,h,functions,length,j):
  ks = list()
  valtemp = values.copy()
  for k in range(4):
    print()
    ktemp = list()
    for i in range(length):
      if k==0:
        pass
      elif k==1 or k==2:
        for index, key in enumerate(values):
          if key!='t':
            value = values.get(key)
            #print(f'{value}+{h/2}({ks[-1][index-1]})')
            valtemp.update({key:truncate(value+(h/2)*ks[-1][index-1])})
          else:
            value = values.get(key)
            valtemp.update({key:truncate(value+(h/2))})
      elif k==3:
        for index, key in enumerate(values):
          if key!='t':
            value = values.get(key)
            valtemp.update({key:truncate(value+h*ks[-1][index-1])})
          else:
            value = values.get(key)
            valtemp.update({key:truncate(value+h)})
      ans,string = functions[i](valtemp)
      ans = truncate(ans)
      strcopy = string
      for key,value in valtemp.items():
        strcopy = strcopy.replace(key,f'({value})')
      print(f'k{k+1},{i+1} = {string} = {strcopy} = {ans}')
      ktemp.append(ans)

    ks.append(ktemp)
  print()
  return ks

def fourthorder(fx, symbs, values, interval, h):
  a,b = interval
  N = round((b-a)/h)
  length = len(symbs)
  
  ts, us, fy, mtrx, functions = printinitials(a,b,h,length, N)
  printmaineq(length)

  temp = truncate(h/2)
  for j in range(N):
    printt(f'j={j})')
    for i in range(1,length+1):
      printt(f'u{i},{j+1} = u{i},{j} + ({h}/6)*(k{i},{j+1} + 2(k{i},{j+1}+k{i},{j+1}) + k{i},{j+1} )',color='green')
      values = {'t':ts[j]}
      for k in range(length):
        values.update({f'u{k+1}':mtrx[j][k]})

    ks = getks(values,h,functions,length,j)

    for i in range(1,length+1):
      mtrx[j+1][i-1] = truncate(mtrx[j][i-1] + (h/6)*(ks[0][i-1] + 2*(ks[1][i-1]+ks[2][i-1]) + ks[3][i-1]))
      printt(f'u{i},{j+1} = {mtrx[j][i-1]} + ({h}/6)*({ks[0][i-1]} + 2({ks[1][i-1]} + {ks[2][i-1]}) + {ks[3][i-1]} ) = {mtrx[j+1][i-1]}',color='green')
    printt('-------------------------------------------------------------')

  printt('Solution','blue')
  printmtrx(mtrx,ts,symbs)
  printt('-------------------------------------------------------------')
  return mtrx, ts

In [13]:
def comparefx(mtrx, realfx, symbs, t, ts):
  printt('\nTrue Values')
  realfxs = [realfx]
  for i in range(len(symbs)):
    apos = "'"*i
    if i!=0:
      realfxs.append(diff(realfxs[-1],t,1))
    print(f'y{apos}(t) = {realfxs[-1]}')
  print()
  
  for j in range(1,len(ts)):
    for i in range(len(symbs)):
      apos = "'"*i
      val = truncate(realfxs[i].subs(t,ts[j]))
      ans = truncate(abs(val - mtrx[j][i]))
      print(f'| y{apos}({ts[j]}) - u{i+1},{j+1} |  = | {val} - {mtrx[j][i]} | = {ans}')
    print()

In [14]:
y1, y, t = symbols("y' y t")              # set symbols
symbs = [y, y1]                           # set symbols
fx = t*ln(t) + (2/t)*y1 - 2*y/(t**2)      # set equation
values = [1,(1,0)]                        # y(1)=1, y'(1)=0
interval = (1,2)                          # set interval
h = 1                                     # set h (1 if none given)

mtrx, ts = fourthorder(fx, symbs, values, interval, h)

realfx = (7/4)*t + (1/2)*(t**3)*ln(t) - (3/4)*(t**3) # set real equation if needed to compare
comparefx(mtrx, realfx, symbs, t, ts)

h = (b-a)/N =>	N = (2-1)/1 = 1
t0 = 1 , t1 = 2 , 
y'' = t*log(t) + 2*y'/t - 2*y/t**2

u0(t) = y(t) , u1(t) = y'(t) , 
y = u1 , y' = u2 , 
u2' = t*log(t) + 2*u2/t - 2*u1/t**2

f1[t, 'u1', 'u2'] = u2
f2[t, 'u1', 'u2'] = t*log(t) + 2*u2/t - 2*u1/t**2

u1,0 = u1(t0) = u1(1) = 1
u2,0 = u2(t0) = u2(1) = 0

ui,j+1  = ui,j + (h/6)*(k1,i + 2(k2,i+k3,i) + k4,i)
where k1,i = fi(tj; u1,j ; u2,j )
      k2,i = fi(tj+h/2; u1,j+(h/2)*k1,1 ; u2,j+(h/2)*k1,2 )
      k3,i = fi(tj+h/2; u1,j+(h/2)*k2,1 ; u2,j+(h/2)*k2,2 )
      k4,i = fi(tj+h/2, u1,j+h*k3,1 , u2,j+h*k3,2 )

j=0)
u1,1 = u1,0 + (1/6)*(k1,1 + 2(k1,1+k1,1) + k1,1 )
u2,1 = u2,0 + (1/6)*(k2,1 + 2(k2,1+k2,1) + k2,1 )

k1,1 = u2 = (0) = 0
k1,2 = t*log(t) + 2*u2/t - 2*u1/t**2 = (1)*log((1)) + 2*(0)/(1) - 2*(1)/(1)**2 = -2

k2,1 = u2 = (-1) = -1
k2,2 = t*log(t) + 2*u2/t - 2*u1/t**2 = (1.5)*log((1.5)) + 2*(-1)/(1.5) - 2*(1)/(1.5)**2 = -1.614

k3,1 = u2 = (-0.807) = -0.807
k3,2 = t*log(t) + 2*u2/t - 2*u1/t**2 = (1.5)*log((1.5)) + 2*(-0.807)/(1.5) - 2